In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table, QTable, hstack
from myspace import MySpace
from sklearn.mixture import GaussianMixture


In [2]:
import sklearn
import jax
import numpy
import scipy

print('scikit-learn', sklearn.__version__)
print('jax', jax.__version__)
print('numpy', numpy.__version__)
print('scipy', scipy.__version__)

#Output:
#scikit-learn 0.23.2
#jax 0.2.5
#numpy 1.19.1
#scipy 1.5.0

scikit-learn 0.23.2
jax 0.2.6
numpy 1.17.2
scipy 1.3.1


In [3]:
gaia = QTable.read('../data/RV-all-result.fits', format='fits')

In [4]:
from zero_point import zpt
zpt.load_tables()

gmag = gaia['phot_g_mean_mag'].value
nueffused = gaia['nu_eff_used_in_astrometry'].value
psc = gaia['pseudocolour'].value
sinbeta = np.sin(np.deg2rad(gaia['ecl_lat'].value))
soltype = gaia['astrometric_params_solved']

zpvals = zpt.get_zpt(gmag, nueffused, psc, sinbeta, soltype)
cparallax=gaia['parallax'].value-zpvals

/usr/local/lib/python3.7/site-packages/zero_point/zpt.py:203: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(phot_g_mean_mag >= 21) or np.any(phot_g_mean_mag <= 6):
/usr/local/lib/python3.7/site-packages/zero_point/zpt.py:203: RuntimeWarning: invalid value encountered in less_equal
  if np.any(phot_g_mean_mag >= 21) or np.any(phot_g_mean_mag <= 6):
/usr/local/lib/python3.7/site-packages/zero_point/zpt.py:208: UserWarning: The apparent magnitude of one or more of the sources is outside the expected range (6-21 mag). 
                Outside this range, there is no further interpolation, thus the values at 6 or 21 are returned.
  UserWarning)
/usr/local/lib/python3.7/site-packages/zero_point/zpt.py:223: UserWarning: The nu_eff_used_in_astrometry of some of the 5p source(s) is outside the expected range (1.1-1.9 
                mag). Outside this range, the zero-point calculated can be seriously wrong.
  UserWarning)
/usr/local/lib/python3.7/site-packages/zero_poi

In [5]:
qindx=(gaia['parallax_over_error']>4.)


In [6]:
def make_anim_xv(XX,VV,tensorsx,myspacex,tensorsxv,myspacexv,gs=150):
    
    
    _cyl = gal.represent_as('cylindrical')
    mask2_r500 = (_cyl.rho < 500*u.pc) & (np.abs(_cyl.z) < 500*u.pc) & (_cyl.rho > 200*u.pc)
    disk_vmask2=(np.sqrt(VV[:,0]**2+VV[:,1]**2+VV[:,2]**2)<200.)
    
    rindx=(disk_vmask2)*(mask2_r500)
    for i in range(0,36):
        wedgedex=rindx*(_cyl.phi+np.pi*u.rad>(i*np.pi/18.)*u.rad)*(_cyl.phi+np.pi*u.rad<((i+3)*np.pi/18.)*u.rad)
        if i==34:
            wedgedex=rindx*(_cyl.phi+np.pi*u.rad>(i*np.pi/18.)*u.rad)*(_cyl.phi+np.pi*u.rad<((i+3)*np.pi/18.)*u.rad)+rindx*(_cyl.phi+np.pi*u.rad>0.)*(_cyl.phi+np.pi*u.rad<((1)*np.pi/18.)*u.rad)
        if i==35:
            wedgedex=rindx*(_cyl.phi+np.pi*u.rad>(i*np.pi/18.)*u.rad)*(_cyl.phi+np.pi*u.rad<((i+3)*np.pi/18.)*u.rad)+rindx*(_cyl.phi+np.pi*u.rad>0.)*(_cyl.phi+np.pi*u.rad<((2)*np.pi/18.)*u.rad)
        print(wedgedex.sum(),'stars in wedge',i)

        fixx=myspacex.get_model_v(tensorsx,VV[wedgedex],XX[wedgedex])
        fixxv=myspacexv.get_model_v(tensorsxv,VV[wedgedex],XX[wedgedex])

        f, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(15,15))
        ax1.hist2d(XX[:,0][wedgedex],XX[:,1][wedgedex],range=[[-500,500],[-500.,500.]],bins=gs,cmin=1.0e-50,rasterized=True,density=True)
        ax1.set_xlabel(r'$X\ (\mathrm{kpc})$',fontsize=20)
        ax1.set_ylabel(r'$Y\ (\mathrm{kpc})$',fontsize=20)
        ax1.set_xlim(-500.,500.)
        ax1.set_ylim(-500.,500.)
        ax1.set_title(r'$\mathrm{Selected\ area}$',fontsize=20)
        ax2.hist2d(VV[:,0][wedgedex],VV[:,1][wedgedex],range=[[-125,125],[-125,125]],bins=gs,cmin=1.0e-50,rasterized=True,density=True)
        ax2.set_xlabel(r'$v_X\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        ax2.set_ylabel(r'$v_Y\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        ax2.set_xlim(-125,125)
        ax2.set_ylim(-125,125)
        ax2.set_title(r'$\mathrm{No\ correction}$',fontsize=20)
        ax3.hist2d(fixx[:,0],fixx[:,1],range=[[-125,125],[-125,125]],bins=gs,cmin=1.0e-50,rasterized=True,density=True)
        ax3.set_xlabel(r'$v_X\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        ax3.set_ylabel(r'$v_Y\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        ax3.set_xlim(-125,125)
        ax3.set_ylim(-125,125)
        ax3.set_title(r'$\mathrm{x\ correction}$',fontsize=20)
        ax4.hist2d(fixxv[:,0],fixxv[:,1],range=[[-125,125],[-125,125]],bins=gs,cmin=1.0e-50,rasterized=True,density=True)
        ax4.set_xlabel(r'$v_X\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        ax4.set_ylabel(r'$v_Y\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        ax4.set_xlim(-125,125)
        ax4.set_ylim(-125,125)
        ax4.set_title(r'$\mathrm{xv\ correction}$',fontsize=20)
        ax1.tick_params(axis='both', which='major', labelsize=15)
        ax2.tick_params(axis='both', which='major', labelsize=15)
        ax3.tick_params(axis='both', which='major', labelsize=15)
        ax4.tick_params(axis='both', which='major', labelsize=15)
        plt.savefig('gaiao'+str("{:02d}".format(i))+'.pdf',bbox_inches='tight')
        plt.close()

    os.system('convert -delay 5 -loop 0 gaiao*.pdf orders.gif')

In [7]:
def make_anim_justx(XX,VV,tensorsx,myspacex,gs=150):
    
    
    _cyl = gal.represent_as('cylindrical')
    mask2_r500 = (_cyl.rho < 500*u.pc) & (np.abs(_cyl.z) < 500*u.pc) & (_cyl.rho > 200*u.pc)
    disk_vmask2=(np.sqrt(VV[:,0]**2+VV[:,1]**2+VV[:,2]**2)<100.)
    
    rindx=(disk_vmask2)*(mask2_r500)
    for i in range(0,36):
        wedgedex=rindx*(_cyl.phi+np.pi*u.rad>(i*np.pi/18.)*u.rad)*(_cyl.phi+np.pi*u.rad<((i+3)*np.pi/18.)*u.rad)
        if i==34:
            wedgedex=rindx*(_cyl.phi+np.pi*u.rad>(i*np.pi/18.)*u.rad)*(_cyl.phi+np.pi*u.rad<((i+3)*np.pi/18.)*u.rad)+rindx*(_cyl.phi+np.pi*u.rad>0.)*(_cyl.phi+np.pi*u.rad<((1)*np.pi/18.)*u.rad)
        if i==35:
            wedgedex=rindx*(_cyl.phi+np.pi*u.rad>(i*np.pi/18.)*u.rad)*(_cyl.phi+np.pi*u.rad<((i+3)*np.pi/18.)*u.rad)+rindx*(_cyl.phi+np.pi*u.rad>0.)*(_cyl.phi+np.pi*u.rad<((2)*np.pi/18.)*u.rad)
        print(wedgedex.sum(),'stars in wedge',i)

        fixx=myspacex.get_model_v(tensorsx,VV[wedgedex],XX[wedgedex])


        f, ((ax1,ax2,ax3)) = plt.subplots(1, 3, figsize=(21,7))
        ax1.hist2d(XX[:,0][wedgedex],XX[:,1][wedgedex],range=[[-0.5,0.5],[-0.5,0.5]],bins=gs,cmin=1.0e-50,rasterized=True,density=True)
        ax1.set_xlabel(r'$X\ (\mathrm{kpc})$',fontsize=20)
        ax1.set_ylabel(r'$Y\ (\mathrm{kpc})$',fontsize=20)
        ax1.set_xlim(-0.5,0.5)
        ax1.set_ylim(-0.5,0.5)
        ax1.set_title(r'$\mathrm{Selected\ area}$',fontsize=20)
        ax2.hist2d(VV[:,0][wedgedex],VV[:,1][wedgedex],range=[[-125,125],[-125,125]],bins=gs,cmin=1.0e-50,rasterized=True,density=True)
        ax2.set_xlabel(r'$v_X\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        #ax2.set_ylabel(r'$v_Y\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        ax2.set_xlim(-125,125)
        ax2.set_ylim(-125,125)
        ax2.set_title(r'$\mathrm{No\ correction}$',fontsize=20)
        ax3.hist2d(fixx[:,0],fixx[:,1],range=[[-125,125],[-125,125]],bins=gs,cmin=1.0e-50,rasterized=True,density=True)
        ax3.set_xlabel(r'$v_X\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        #ax3.set_ylabel(r'$v_Y\ (\mathrm{km\ s}^{-1})$',fontsize=20)
        ax3.set_xlim(-125,125)
        ax3.set_ylim(-125,125)
        ax3.set_title(r'$\mathrm{x\ correction}$',fontsize=20)

        
        ax1.tick_params(axis='both', which='major', labelsize=15)

        ax3.tick_params(axis='both', which='major', labelsize=15)

        plt.savefig('93-'+str("{:02d}".format(i))+'.pdf',bbox_inches='tight')
        plt.close()

    os.system('convert -delay 5 -loop 0 93-*.pdf justx.gif')

In [8]:
c = coord.SkyCoord(ra=gaia['ra'][qindx],dec=gaia['dec'][qindx],distance=1./cparallax[qindx]*u.kpc,pm_ra_cosdec=gaia['pmra'][qindx],pm_dec=gaia['pmdec'][qindx],radial_velocity=gaia['radial_velocity'][qindx])



/usr/local/lib/python3.7/site-packages/astropy/coordinates/distances.py:181: RuntimeWarning: invalid value encountered in less
  if not allow_negative and np.any(distance.value < 0):


In [9]:
gal = c.galactic
gal.set_representation_cls('cartesian')

In [10]:
xyz = np.vstack((gal.u.to(u.kpc).value, 
                 gal.v.to(u.kpc).value,
                 gal.w.to(u.kpc).value)).T

UVW = np.vstack((gal.U.to(u.km/u.s).value, 
                 gal.V.to(u.km/u.s).value, 
                 gal.W.to(u.km/u.s).value)).T

disk_vmask = np.linalg.norm(UVW, axis=1) < 150.

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in less
  if __name__ == '__main__':


In [11]:
XX=xyz
VV=UVW
dist2=np.sqrt(XX[:,0]**2+XX[:,1]**2)

_cyl = gal.represent_as('cylindrical')
mask_r100 = (_cyl.rho < 100*u.pc) & (np.abs(_cyl.z) < 150*u.pc)
mask_r300 = (_cyl.rho < 300*u.pc) & (np.abs(_cyl.z) < 500*u.pc)
mask_r500 = (_cyl.rho < 500*u.pc) & (np.abs(_cyl.z) < 500*u.pc)
mask_r100.sum(), mask_r500.sum()
#local_mask=(dist2<0.2)*(np.fabs(XX[:,2])<0.2)
#train_mask=(dist2>0.2)*(dist2<0.5)*(np.fabs(XX[:,2])<0.5)

local_v = UVW[disk_vmask & mask_r100]
local_x = xyz[disk_vmask & mask_r100]

print(local_v.shape)

(98166, 3)


/usr/local/lib/python3.7/site-packages/astropy/units/quantity.py:464: RuntimeWarning: invalid value encountered in less
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [12]:
local_gmm = GaussianMixture(n_components=64)
local_gmm.fit(local_v)

GaussianMixture(n_components=64)

In [13]:
# Just X
# myspace = MySpace(local_gmm, terms=['x'])

In [14]:
myspacexv = MySpace(local_gmm, terms=['x','xv'])
# myspacexvx = MySpace(local_gmm, terms=['x','xv','xx'])

In [15]:

train_v = UVW[disk_vmask & mask_r300]
train_x = xyz[disk_vmask & mask_r300]

test_v = UVW[disk_vmask & mask_r500]
test_x = xyz[disk_vmask & mask_r500]

local_v.shape, train_v.shape, test_v.shape

((98166, 3), (829927, 3), (1499237, 3))

In [16]:
# res, tensors = myspace.fit(train_x, train_v)

In [ ]:
resxv, tensorsxv = myspacexv.fit(train_x, train_v)
# resxvx, tensorsxvx = myspacexvx.fit(train_x, train_v)
print(tensors)

In [ ]:
def calculate_oort_constants(tensors):
    Aij=tensors['Aij']
    A = 0.5 * (-1.*Aij[0, 1] + -1.*Aij[1, 0])
    B = 0.5 * (-1.*Aij[1, 0] - -1.*Aij[0, 1])
    K = 0.5 * (-1.*Aij[0, 0] + -1.*Aij[1, 1])
    C = 0.5 * (-1.*Aij[0, 0] - -1.*Aij[1, 1])
    print('A=',A,'B=',B,'C=',C,'K=',K)
    return(A,B,C,K)
calculate_oort_constants(tensors)

In [ ]:
fixx=myspace.get_model_v(tensors,test_v,test_x)

In [ ]:
fixxv=myspacexv.get_model_v(tensorsxv,test_v,test_x)
fixxvx=myspacexvx.get_model_v(tensorsxvx,test_v,test_x)

In [ ]:
f, ((ax1,ax2)) = plt.subplots(1, 2, figsize=(10,5))
gs=200
ax1.hexbin(test_v[:,0],test_v[:,1],extent=[-125,125,-125,125],mincnt=1,rasterized=True,gridsize=gs)
ax1.set_title('Uncorrected',fontsize=20)
ax1.set_xlabel('vx (km/s)',fontsize=20)
ax1.set_ylabel('vy (km/s)',fontsize=20)
ax1.set_xlim(-125,125)
ax1.set_ylim(-125,125)
ax2.hexbin(fixx[:,0],fixx[:,1],extent=[-125,125,-125,125],mincnt=1,rasterized=True,gridsize=gs)
ax2.set_title('x corrected',fontsize=20)
ax2.set_xlabel('vx (km/s)',fontsize=20)
ax2.set_xlim(-125,125)
ax2.set_ylim(-125,125)
plt.show()

In [ ]:
make_anim_justx(XX,VV,tensors,myspace)

In [ ]:
make_anim_xv(XX,VV,tensors,myspace,tensorsxv,myspacexv)

In [ ]:
f, ((ax1,ax2,ax3,ax4)) = plt.subplots(1, 4, figsize=(20,5))
gs=200
ax1.hexbin(test_v[:,0],test_v[:,1],extent=[-125,125,-125,125],mincnt=1,rasterized=True,gridsize=gs)
ax1.set_title('Uncorrected',fontsize=20)
ax1.set_xlabel('vx (km/s)',fontsize=20)
ax1.set_ylabel('vy (km/s)',fontsize=20)
ax1.set_xlim(-125,125)
ax1.set_ylim(-125,125)
ax2.hexbin(fixx[:,0],fixx[:,1],extent=[-125,125,-125,125],mincnt=1,rasterized=True,gridsize=gs)
ax2.set_title('x corrected',fontsize=20)
ax2.set_xlabel('vx (km/s)',fontsize=20)
ax2.set_xlim(-125,125)
ax2.set_ylim(-125,125)
ax3.hexbin(fixxv[:,0],fixxv[:,1],extent=[-125,125,-125,125],mincnt=1,rasterized=True,gridsize=gs)
ax3.set_title('xv corrected',fontsize=20)
ax3.set_xlabel('vx (km/s)',fontsize=20)
ax3.set_xlim(-125,125)
ax3.set_ylim(-125,125)
ax4.hexbin(fixxvx[:,0],fixxvx[:,1],extent=[-125,125,-125,125],mincnt=1,rasterized=True,gridsize=gs)
ax4.set_title('xvxx corrected',fontsize=20)
ax4.set_xlabel('vx (km/s)',fontsize=20)
ax4.set_xlim(-125,125)
ax4.set_ylim(-125,125)
#plt.savefig('comp64.pdf',bbox_inches='tight')
plt.show()